# Data Collation

> How do books become classics?
- toc: true 
- badges: true
- comments: true
- categories: [canon]

## How can I access a huge dataset?

Some of my data, specifically the goodreads dataset and the Amazon sales ranks, are in huge JSON files that need to be converted to an appropriate ontology for a book. Loading this all into memory at once will crash my computer.

In [1]:
import pandas as pd

pd.set_option("display.max_columns", None)


In [ ]:
books_json = pd.read_json('/run/media/mage/INDESTRUCTIBLESLIME/Replaceable/datasets/goodreads/downloads/goodreads_books.json', lines=True, chunksize=200)

In [ ]:
for chunk in books_json:
    break

In [ ]:
chunk

In [ ]:
chunk.loc[395]

I can move through it in chunks and save each piece as a separate .csv file, then process that data more easily. And in the process, I might as well filter out any books I definitely don't want -- like ebooks.

In [ ]:
chunk['is_ebook'].value_counts()

In [ ]:
# %rm records/*

In [2]:
from tqdm import tqdm

In [ ]:
books_json = pd.read_json('/run/media/mage/INDESTRUCTIBLESLIME/Replaceable/datasets/goodreads/downloads/goodreads_books.json', lines=True, chunksize=100000)

In [ ]:
for i, chunk in tqdm(enumerate(books_json)):
    filtered_chunk = chunk[chunk['is_ebook'] == 'false']
    filtered_chunk.to_csv(f'records/goodreads_books_{str(i).zfill(4)}.csv')
#     print(i, len(filtered_chunk))

Over **TWO MILLION BOOKS** in this dataset! How exciting.

Now we need to do something with them. Specifically, it would be smart to look at a sample chunk and clean it up, then run that process across all the chunks. Then hopefully it will be small enough to hold in memory at once, and we can start doing transforms.

In [ ]:
df = pd.read_csv('records/goodreads_books_0007.csv')

In [3]:
from pandas_profiling import ProfileReport

In [ ]:
profile = ProfileReport(df, title="Pandas Profiling Report")


In [ ]:
profile.to_widgets()

Takeaways from the profile:

Columns to remove:
- with more than 50% missing values:
    ```
    edition_information
    kindle_asin
    ````
- with constant values:
    ```
    country_code
    is_ebook
    ````
    
The `asin` values are mostly missing, but we're going to need those to compare to Amazon data later so keep that column.

Lots of missing values for `isbn` and `isbn13`, maybe they will cancel each other out?

The `average_rating` is around 3.8, with a standard deviation around 0.53. A 4.57 rating is 95th percentile, whcih might be important to know later. Almost no books have 0, 1, or 2 ratings, so the 5th percentile is right at 3.00.

Formats: still have audiobooks in here. Probably better to clean them out actively and leave weird one-off data in there, rtather than restrict only to known book formats.

The `publication_month` might be useful, but not `publication_day` because too mny values are missing. Unfortunately `publication_year` is missing 15% of it's 

I am not really qualified to comment on th popularity of non-english books, but the `lang` field is missing a lot of values, so I'll keep all those books until I find a compelling reason to leave them out. But should probably filter out all the non-English-lang rows..

Only about half of books have a `similar_books` measurement, but that seems like ti will be really useful so I'll keep them all.

The `num_pages` distribution looks good, just skewed by a few multi-volume sets. That's fine for our purposes.

It looks like `title_without_series` actually includes the series name anyway, so let's just drop that entirely. 

After that much cleanup, the dataset should be a little smaller and then we can really explore the whole terrain.

In [ ]:
dropped_df = df.drop(['edition_information','kindle_asin', 'country_code', 'is_ebook', 'publication_day', 'title_without_series'], axis=1)

In [ ]:
langs = [o for o in df['language_code'].value_counts().index if o not in ['eng','en-US','en-GB','en-CA']]

In [ ]:
eng_df = dropped_df[~dropped_df['language_code'].isin(langs)]
len(eng_df)

In [ ]:
formats = ['Audio CD', 'Audiobook', 'Audible Audio', 'Audio', 'audio cd', 'Audio Cassette', 'MP3 CD', 'CD-ROM', 'Podiobook', 'Audio Play', 'MP3', 'audio']

paper_df = eng_df[~eng_df['format'].isin(formats)]

In [ ]:
len(paper_df)

In [ ]:
def clean_df(df):
    
    dropped_df = df.drop(['edition_information','kindle_asin', 'country_code', 'is_ebook', 'publication_day', 'title_without_series'], axis=1)
    eng_df = dropped_df[~dropped_df['language_code'].isin(langs)]
    paper_df = eng_df[~eng_df['format'].isin(formats)]
    return(paper_df)

In [ ]:
cleaned_df = clean_df(df)
len(cleaned_df)

Seems to work. Let's run it over each one sequentially, appending it to an overall dataframe from inside a context manager


## How can I collate the data into an accessible format?

In [ ]:
total_df = pd.DataFrame()

In [ ]:
for i in tqdm(range(24)):
    df = pd.read_csv(f'records/goodreads_books_{str(i).zfill(4)}.csv')
    cleaned_df = clean_df(df)
    print(len(cleaned_df), len(total_df))
    total_df = pd.concat([total_df,cleaned_df], ignore_index=True)

In [ ]:
total_df

In [ ]:
total_df.to_csv('records/cleaned_goodreads_books.csv')

# Explore


## Find top genre for each book

In [3]:
total_df = pd.read_csv('records/cleaned_goodreads_books.csv')

In [4]:
total_df

,Unnamed: 0,Unnamed: 0.1,isbn,text_reviews_count,series,language_code,popular_shelves,asin,average_rating,similar_books,description,format,link,authors,publisher,num_pages,isbn13,publication_month,publication_year,url,image_url,book_id,ratings_count,work_id,title
0,0,0,0312853122,1.0,[],NaN,"[{'count': '3', 'name': 'to-read'}, {'count': ...",NaN,4.00,[],NaN,Paperback,https://www.goodreads.com/book/show/5333265-w-...,"[{'author_id': '604031', 'role': ''}]",St. Martin's Press,256.0,9780312853129,9.0,1984.0,https://www.goodreads.com/book/show/5333265-w-...,https://images.gr-assets.com/books/1310220028m...,5333265,3.0,5400751.0,W.C. Fields: A Life on Film
1,1,2,NaN,7.0,['189911'],eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,4.03,"['19997', '828466', '1569323', '425389', '1176...",Omnibus book club edition containing the Ladie...,Hardcover,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]","Nelson Doubleday, Inc.",600.0,NaN,NaN,1987.0,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140.0,8948723.0,"The Unschooled Wizard (Sun Wolf and Starhawk, ..."
2,2,3,0743294297,3282.0,[],eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",NaN,3.49,"['6604176', '6054190', '2285777', '82641', '75...",Addie Downs and Valerie Adler were eight when ...,Hardcover,https://www.goodreads.com/book/show/6066819-be...,"[{'author_id': '9212', 'role': ''}]",Atria Books,368.0,9780743294294,7.0,2009.0,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184.0,6243154.0,Best Friends Forever
3,3,4,0850308712,5.0,[],NaN,"[{'count': '32', 'name': 'to-read'}, {'count':...",NaN,3.40,[],NaN,NaN,https://www.goodreads.com/book/show/287140.Run...,"[{'author_id': '149918', 'role': ''}]",NaN,NaN,9780850308716,NaN,NaN,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,287140,15.0,278577.0,Runic Astrology: Starcraft and Timekeeping in ...
4,4,5,1599150603,7.0,[],NaN,"[{'count': '56', 'name': 'to-read'}, {'count':...",NaN,4.13,[],"Relates in vigorous prose the tale of Aeneas, ...",Paperback,https://www.goodreads.com/book/show/287141.The...,"[{'author_id': '3041852', 'role': ''}]",Yesterday's Classics,162.0,9781599150604,9.0,2006.0,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,46.0,278578.0,The Aeneid for Boys and Girls
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216067,1216067,2360645,0689852959,1.0,[],NaN,"[{'count': '22', 'name': 'to-read'}, {'count':...",NaN,4.36,[],One of the most popular series ever published ...,Paperback,https://www.goodreads.com/book/show/331839.Jac...,"[{'author_id': '10681', 'role': ''}, {'author_...",Aladdin,176.0,9780689852954,9.0,2002.0,https://www.goodreads.com/book/show/331839.Jac...,https://s.gr-assets.com/assets/nophoto/book/11...,331839,18.0,25313618.0,Jacqueline Kennedy Onassis: Friend of the Arts
1216068,1216068,2360647,0373126476,9.0,[],NaN,"[{'count': '78', 'name': 'to-read'}, {'count':...",NaN,3.42,"['2200344', '695337', '10333421', '1934240', '...","Blackmailed into marriage to save her family, ...",Paperback,https://www.goodreads.com/book/show/2685097-th...,"[{'author_id': '319441', 'role': ''}]",Harlequin,192.0,9780373126477,7.0,2007.0,https://www.goodreads.com/book/show/2685097-th...,https://s.gr-assets.com/assets/nophoto/book/11...,2685097,112.0,2710420.0,The Spaniard's Blackmailed Bride
1216069,1216069,2360651,178092870X,2.0,[],eng,"[{'count': '702', 'name': 'to-read'}, {'count'...",NaN,3.50,"['12064253', '25017213', '571796', '27306126',...",Sir Arthur Conan Doyle is brought back to life...,Paperback,https://www.goodreads.com/book/show/26168430-s...,"[{'author_id': '2448', 'role': ''}, {'author_i...",MX Publishing,148.0,9781780928708,8.0,2015.0,https://www.goodreads.com/book/show/261

In [11]:
def get_genre(shelf_dict):
    if len(shelf_dict) > 0:
        top = [o for o in shelf_dict if o['name'] not in ['to-read', 'currently-reading', 'kindle', 'non-fiction', 'nonfiction']]
        if len(top):
            return(top[0]['name'])
        else:
            return('NONFIC')
    return('UNK')

In [12]:
g = (get_genre(eval(o)) for o in total_df['popular_shelves'])
total_df['top_genre'] = pd.Series(g, dtype='unicode')

In [34]:
total_df.tail()

,Unnamed: 0,Unnamed: 0.1,isbn,text_reviews_count,series,language_code,popular_shelves,asin,average_rating,similar_books,description,format,link,authors,publisher,num_pages,isbn13,publication_month,publication_year,url,image_url,book_id,ratings_count,work_id,title,top_genre
1216067,1216067,2360645,0689852959,1.0,[],NaN,"[{'count': '22', 'name': 'to-read'}, {'count':...",NaN,4.36,[],One of the most popular series ever published ...,Paperback,https://www.goodreads.com/book/show/331839.Jac...,"[{'author_id': '10681', 'role': ''}, {'author_...",Aladdin,176.0,9780689852954,9.0,2002.0,https://www.goodreads.com/book/show/331839.Jac...,https://s.gr-assets.com/assets/nophoto/book/11...,331839,18.0,25313618.0,Jacqueline Kennedy Onassis: Friend of the Arts,biography
1216068,1216068,2360647,0373126476,9.0,[],NaN,"[{'count': '78', 'name': 'to-read'}, {'count':...",NaN,3.42,"['2200344', '695337', '10333421', '1934240', '...","Blackmailed into marriage to save her family, ...",Paperback,https://www.goodreads.com/book/show/2685097-th...,"[{'author_id': '319441', 'role': ''}]",Harlequin,192.0,9780373126477,7.0,2007.0,https://www.goodreads.com/book/show/2685097-th...,https://s.gr-assets.com/assets/nophoto/book/11...,2685097,112.0,2710420.0,The Spaniard's Blackmailed Bride,harlequin
1216069,1216069,2360651,178092870X,2.0,[],eng,"[{'count': '702', 'name': 'to-read'}, {'count'...",NaN,3.50,"['12064253', '25017213', '571796', '27306126',...",Sir Arthur Conan Doyle is brought back to life...,Paperback,https://www.goodreads.com/book/show/26168430-s...,"[{'author_id': '2448', 'role': ''}, {'author_i...",MX Publishing,148.0,9781780928708,8.0,2015.0,https://www.goodreads.com/book/show/26168430-s...,https://images.gr-assets.com/books/1440592011m...,26168430,6.0,46130263.0,Sherlock Holmes and the July Crisis,mystery
1216070,1216070,2360652,0765197456,6.0,[],NaN,"[{'count': '37', 'name': 'to-read'}, {'count':...",NaN,4.00,[],"Gathers poems by William Blake, Emily Bronte, ...",Hardcover,https://www.goodreads.com/book/show/2342551.Th...,"[{'author_id': '82312', 'role': 'Editor'}]",Smithmark Publishers,96.0,9780765197450,8.0,1996.0,https://www.goodreads.com/book/show/2342551.Th...,https://s.gr-assets.com/assets/nophoto/book/11...,2342551,36.0,2349247.0,The Children's Classic Poetry Collection,poetry
1216071,1216071,2360653,162378140X,17.0,['658195'],eng,"[{'count': '56', 'name': 'to-read'}, {'count':...",NaN,4.37,"['23562786', '13548289', '26094541', '20570173...","Volume One contains: ""Claimed,"" ""Tainted,"" and...",Paperback,https://www.goodreads.com/book/show/22017381-1...,"[{'author_id': '7789809', 'role': ''}]",Guerrilla Wordfare,306.0,9781623781408,4.0,2014.0,https://www.goodreads.com/book/show/22017381-1...,https://images.gr-assets.com/books/1398621236m...,22017381,70.0,41332799.0,"101 Nights: Volume One (101 Nights, #1-3)",erotica


In [37]:
total_df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

,isbn,text_reviews_count,series,language_code,popular_shelves,asin,average_rating,similar_books,description,format,link,authors,publisher,num_pages,isbn13,publication_month,publication_year,url,image_url,book_id,ratings_count,work_id,title,top_genre
0,0312853122,1.0,[],NaN,"[{'count': '3', 'name': 'to-read'}, {'count': ...",NaN,4.00,[],NaN,Paperback,https://www.goodreads.com/book/show/5333265-w-...,"[{'author_id': '604031', 'role': ''}]",St. Martin's Press,256.0,9780312853129,9.0,1984.0,https://www.goodreads.com/book/show/5333265-w-...,https://images.gr-assets.com/books/1310220028m...,5333265,3.0,5400751.0,W.C. Fields: A Life on Film,p
1,NaN,7.0,['189911'],eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,4.03,"['19997', '828466', '1569323', '425389', '1176...",Omnibus book club edition containing the Ladie...,Hardcover,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]","Nelson Doubleday, Inc.",600.0,NaN,NaN,1987.0,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140.0,8948723.0,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",fantasy
2,0743294297,3282.0,[],eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",NaN,3.49,"['6604176', '6054190', '2285777', '82641', '75...",Addie Downs and Valerie Adler were eight when ...,Hardcover,https://www.goodreads.com/book/show/6066819-be...,"[{'author_id': '9212', 'role': ''}]",Atria Books,368.0,9780743294294,7.0,2009.0,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184.0,6243154.0,Best Friends Forever,chick-lit
3,0850308712,5.0,[],NaN,"[{'count': '32', 'name': 'to-read'}, {'count':...",NaN,3.40,[],NaN,NaN,https://www.goodreads.com/book/show/287140.Run...,"[{'author_id': '149918', 'role': ''}]",NaN,NaN,9780850308716,NaN,NaN,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,287140,15.0,278577.0,Runic Astrology: Starcraft and Timekeeping in ...,runes
4,1599150603,7.0,[],NaN,"[{'count': '56', 'name': 'to-read'}, {'count':...",NaN,4.13,[],"Relates in vigorous prose the tale of Aeneas, ...",Paperback,https://www.goodreads.com/book/show/287141.The...,"[{'author_id': '3041852', 'role': ''}]",Yesterday's Classics,162.0,9781599150604,9.0,2006.0,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,46.0,278578.0,The Aeneid for Boys and Girls,history
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216067,0689852959,1.0,[],NaN,"[{'count': '22', 'name': 'to-read'}, {'count':...",NaN,4.36,[],One of the most popular series ever published ...,Paperback,https://www.goodreads.com/book/show/331839.Jac...,"[{'author_id': '10681', 'role': ''}, {'author_...",Aladdin,176.0,9780689852954,9.0,2002.0,https://www.goodreads.com/book/show/331839.Jac...,https://s.gr-assets.com/assets/nophoto/book/11...,331839,18.0,25313618.0,Jacqueline Kennedy Onassis: Friend of the Arts,biography
1216068,0373126476,9.0,[],NaN,"[{'count': '78', 'name': 'to-read'}, {'count':...",NaN,3.42,"['2200344', '695337', '10333421', '1934240', '...","Blackmailed into marriage to save her family, ...",Paperback,https://www.goodreads.com/book/show/2685097-th...,"[{'author_id': '319441', 'role': ''}]",Harlequin,192.0,9780373126477,7.0,2007.0,https://www.goodreads.com/book/show/2685097-th...,https://s.gr-assets.com/assets/nophoto/book/11...,2685097,112.0,2710420.0,The Spaniard's Blackmailed Bride,harlequin
1216069,178092870X,2.0,[],eng,"[{'count': '702', 'name': 'to-read'}, {'count'...",NaN,3.50,"['12064253', '25017213', '571796', '27306126',...",Sir Arthur Conan Doyle is brought back to life...,Paperback,https://www.goodreads.com/book/show/26168430-s...,"[{'author_id': '2448', 'role': ''}, {'author_i...",MX Publishing,148.0,9781780928708,8.0,2015.0,https://www.goodreads.com/book/show/26168430-s...,https://images.gr-ass

In [38]:
total_df.to_csv('records/cleaned_goodreads_books.csv')

In [3]:
total_df = pd.read_csv('records/cleaned_goodreads_books.csv')

## Get author data

It's in a different JSON file, we'll have to copy the data in somehow

In [4]:
author_df = pd.read_json('/run/media/mage/INDESTRUCTIBLESLIME/Replaceable/datasets/goodreads/downloads/goodreads_book_authors.json', lines=True)

In [5]:
author_df = author_df.set_index('author_id')

In [6]:
author_df

,average_rating,text_reviews_count,name,ratings_count
author_id,,,,
604031,3.98,7,Ronald J. Fields,49
626222,4.08,28716,Anita Diamant,546796
10333,3.92,5075,Barbara Hambly,122118
9212,3.68,36262,Jennifer Weiner,888522
149918,3.82,96,Nigel Pennick,1740
...,...,...,...,...
197551,4.36,4,Patty Furbush,11
3988103,4.33,3,Jim Schlinkman,6
13464507,4.00,2,Rich Jolly,18


In [7]:
total_df['authors']

0                      [{'author_id': '604031', 'role': ''}]
1                       [{'author_id': '10333', 'role': ''}]
2                        [{'author_id': '9212', 'role': ''}]
3                      [{'author_id': '149918', 'role': ''}]
4                     [{'author_id': '3041852', 'role': ''}]
                                 ...                        
1216067    [{'author_id': '10681', 'role': ''}, {'author_...
1216068                [{'author_id': '319441', 'role': ''}]
1216069    [{'author_id': '2448', 'role': ''}, {'author_i...
1216070           [{'author_id': '82312', 'role': 'Editor'}]
1216071               [{'author_id': '7789809', 'role': ''}]
Name: authors, Length: 1216072, dtype: object

In [8]:
author_df.loc[604031]

average_rating                    3.98
text_reviews_count                   7
name                  Ronald J. Fields
ratings_count                       49
Name: 604031, dtype: object

In [40]:
def get_author(a_list):
    try:
        top = int(eval(a_list)[0]['author_id'])
        return(author_df.loc[top]['name'])
    except IndexError as e:
        print(e)
        return('UNK')


In [42]:
from tqdm import tqdm

In [43]:
g = [get_author(a_list) for a_list in tqdm(total_df['authors'])]


  3%|▎         | 31308/1216072 [00:04<02:32, 7771.24it/s]








 14%|█▍        | 170558/1216072 [00:22<02:14, 7792.28it/s]A

list index out of range


 27%|██▋       | 332791/1216072 [00:42<01:53, 7754.12it/s]

list index out of range


 34%|███▍      | 415285/1216072 [00:53<01:42, 7836.99it/s]

...

 55%|█████▍    | 663032/1216072 [01:25<01:14, 7387.21it/s]

list index out of range


 96%|█████████▌| 1163852/1216072 [02:30<00:06, 7686.75it/s]

list index out of range


100%|██████████| 1216072/1216072 [02:37<00:00, 7721.25it/s]


In [44]:
total_df['author_name'] = g

In [45]:
total_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,isbn,text_reviews_count,series,language_code,popular_shelves,asin,average_rating,similar_books,description,format,link,authors,publisher,num_pages,isbn13,publication_month,publication_year,url,image_url,book_id,ratings_count,work_id,title,top_genre,author_name
0,0,0,0,0312853122,1.0,[],NaN,"[{'count': '3', 'name': 'to-read'}, {'count': ...",NaN,4.00,[],NaN,Paperback,https://www.goodreads.com/book/show/5333265-w-...,"[{'author_id': '604031', 'role': ''}]",St. Martin's Press,256.0,9780312853129,9.0,1984.0,https://www.goodreads.com/book/show/5333265-w-...,https://images.gr-assets.com/books/1310220028m...,5333265,3.0,5400751.0,W.C. Fields: A Life on Film,p,Ronald J. Fields
1,1,1,2,NaN,7.0,['189911'],eng,"[{'count': '58', 'name': 'to-read'}, {'count':...",B00071IKUY,4.03,"['19997', '828466', '1569323', '425389', '1176...",Omnibus book club edition containing the Ladie...,Hardcover,https://www.goodreads.com/book/show/7327624-th...,"[{'author_id': '10333', 'role': ''}]","Nelson Doubleday, Inc.",600.0,NaN,NaN,1987.0,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,7327624,140.0,8948723.0,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",fantasy,Barbara Hambly
2,2,2,3,0743294297,3282.0,[],eng,"[{'count': '7615', 'name': 'to-read'}, {'count...",NaN,3.49,"['6604176', '6054190', '2285777', '82641', '75...",Addie Downs and Valerie Adler were eight when ...,Hardcover,https://www.goodreads.com/book/show/6066819-be...,"[{'author_id': '9212', 'role': ''}]",Atria Books,368.0,9780743294294,7.0,2009.0,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,6066819,51184.0,6243154.0,Best Friends Forever,chick-lit,Jennifer Weiner
3,3,3,4,0850308712,5.0,[],NaN,"[{'count': '32', 'name': 'to-read'}, {'count':...",NaN,3.40,[],NaN,NaN,https://www.goodreads.com/book/show/287140.Run...,"[{'author_id': '149918', 'role': ''}]",NaN,NaN,9780850308716,NaN,NaN,https://www.goodreads.com/book/show/287140.Run...,https://images.gr-assets.com/books/1413219371m...,287140,15.0,278577.0,Runic Astrology: Starcraft and Timekeeping in ...,runes,Nigel Pennick
4,4,4,5,1599150603,7.0,[],NaN,"[{'count': '56', 'name': 'to-read'}, {'count':...",NaN,4.13,[],"Relates in vigorous prose the tale of Aeneas, ...",Paperback,https://www.goodreads.com/book/show/287141.The...,"[{'author_id': '3041852', 'role': ''}]",Yesterday's Classics,162.0,9781599150604,9.0,2006.0,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,287141,46.0,278578.0,The Aeneid for Boys and Girls,history,Alfred J. Church
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1216067,1216067,1216067,2360645,0689852959,1.0,[],NaN,"[{'count': '22', 'name': 'to-read'}, {'count':...",NaN,4.36,[],One of the most popular series ever published ...,Paperback,https://www.goodreads.com/book/show/331839.Jac...,"[{'author_id': '10681', 'role': ''}, {'author_...",Aladdin,176.0,9780689852954,9.0,2002.0,https://www.goodreads.com/book/show/331839.Jac...,https://s.gr-assets.com/assets/nophoto/book/11...,331839,18.0,25313618.0,Jacqueline Kennedy Onassis: Friend of the Arts,biography,Beatrice Gormley
1216068,1216068,1216068,2360647,0373126476,9.0,[],NaN,"[{'count': '78', 'name': 'to-read'}, {'count':...",NaN,3.42,"['2200344', '695337', '10333421', '1934240', '...","Blackmailed into marriage to save her family, ...",Paperback,https://www.goodreads.com/book/show/2685097-th...,"[{'author_id': '319441', 'role': ''}]",Harlequin,192.0,9780373126477,7.0,2007.0,https://www.goodreads.com/book/show/2685097-th...,https://s.gr-assets.com/assets/nophoto/book/11...,2685097,112.0,2710420.0,The Spaniard's Blackmailed Bride,harlequin,Trish Morey
1216069,1216069,1216069,2360651,178092870X,2.0,[],eng,"[{'count': '702', 'name': 'to-read'}, {'count'...",NaN,3.50,"['12064253', '25017213', '571796', '27306126',...",S

In [46]:
total_df.to_csv('records/cleaned_goodreads_books.csv')

In [48]:
total_df.keys()

Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'isbn',
       'text_reviews_count', 'series', 'language_code', 'popular_shelves',
       'asin', 'average_rating', 'similar_books', 'description', 'format',
       'link', 'authors', 'publisher', 'num_pages', 'isbn13',
       'publication_month', 'publication_year', 'url', 'image_url', 'book_id',
       'ratings_count', 'work_id', 'title', 'top_genre', 'author_name'],
      dtype='object')

In [49]:
graph_df = total_df[['title',
                    'author_name',
                    'top_genre',
                    'publisher',
                    'publication_year',
                    'format',
                    'num_pages',
                    'average_rating',
                    'ratings_count',
                    'text_reviews_count',
                    'description',
                   ]]

In [81]:
graph_df['publication_year'] = [g if g < 2021 else 1950 for g in graph_df['publication_year']]

In [82]:
graph_df.to_csv('records/to_graph.csv')